<a href="https://colab.research.google.com/github/gregorywmorris/MLZoom2022/blob/main/Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Churn Guided Project
##Week Three

In [443]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline 

from sklearn.model_selection import train_test_split

In [444]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [445]:
!wget $data -O data.csv

--2022-09-21 20:34:04--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>] 954.59K  --.-KB/s    in 0.03s   

2022-09-21 20:34:05 (31.8 MB/s) - ‘data.csv’ saved [977501/977501]



In [446]:
df = pd.read_csv('data.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [447]:
df.head().T #Use transpose as an alternate way to see columns and data

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [448]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
  df[c] = df[c].str.lower().str.replace(' ', '_')

df.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [449]:
df.dtypes # seniorcitizen is a number (0 or 1) and total charges should be a number

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [450]:
# pd.to_numeric(df.totalcharges) # ValueError: Unable to parse string "_" at position 488

In [451]:
tc = pd.to_numeric(df.totalcharges, errors='coerce')
tc.dtype

dtype('float64')

In [452]:
df[tc.isnull()][['customerid','totalcharges']]

,customerid,totalcharges
488,4472-lvygi,_
753,3115-czmzd,_
936,5709-lvoeq,_
1082,4367-nuyao,_
1340,1371-dwpaz,_
3331,7644-omvmy,_
3826,3213-vvolg,_
4380,2520-sgtta,_
5218,2923-arzlg,_
6670,4075-wkniu,_


In [453]:
df.totalcharges = pd.to_numeric(tc) # Replace
# the totalcharges column with tc
df.totalcharges.dtype

dtype('float64')

In [454]:
df.totalcharges = df.totalcharges.fillna(0)

In [455]:
df.churn.isnull().sum()

0

In [456]:
df.churn.head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [457]:
df.churn = (df.churn == 'yes').astype(int)
# convert yes and no to 0 and 1
df.churn.head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int64

# 3.3 Setting up the Validation Framework

In [458]:
train_test_split?
#help page

In [459]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
len(df_full_train), len(df_test)

(5634, 1409)

In [460]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
len(df_train), len(df_test), len(df_val)

(4225, 1409, 1409)

In [461]:
len(df_train)+ len(df_test)+ len(df_val) == len(df)

True

In [462]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [463]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values


In [464]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

In [465]:
df_full_train.totalcharges.dtype

dtype('float64')

# 3.4 EDA

In [466]:
df_full_train = df_full_train.reset_index(drop=True)

In [467]:
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [468]:
df_full_train

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
1,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
2,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
3,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
4,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,0781-lkxbr,male,1,no,no,9,yes,yes,fiber_optic,no,...,yes,no,yes,yes,month-to-month,yes,electronic_check,100.50,918.60,1
5630,3507-gasnp,male,0,no,yes,60,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.95,1189.90,0
5631,8868-wozgu,male,0,no,no,28,yes,yes,fiber_optic,no,...,yes,no,yes,yes,month-to-month,yes,electronic_check,105.70,2979.50,1
5632,1251-krreg,male,0,no,no,2,yes,yes,dsl,no,...,no,no,no,no,month-to-month,yes,mailed_check,54.40,114.10,1


In [470]:
df_full_train.churn.value_counts()

0    4113
1    1521
Name: churn, dtype: int64

In [471]:
df_full_train.churn.value_counts(normalize=True) # 1 is the churn rate, showing as about 27%

0    0.730032
1    0.269968
Name: churn, dtype: float64

In [472]:
global_churn = df_full_train.churn.mean()
round(global_churn_rate, 2)

0.27

In [473]:
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int64
dtype: object

In [474]:
numerical = ['tenure','monthlycharges', 'totalcharges']
df_full_train[numerical].dtypes

tenure              int64
monthlycharges    float64
totalcharges      float64
dtype: object

In [475]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
        'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [476]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

# 3.5 Feature Importance: Churn Rate and Risk Ratio

**Churn Rate**

In [477]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

0.27682403433476394

In [478]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

0.2632135306553911

In [479]:
global_churn

0.26996805111821087

Neglegable difference in churn if male of female. 

In [480]:
df_full_train.partner.value_counts()

no     2932
yes    2702
Name: partner, dtype: int64

In [481]:
churn_with_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_with_partner

0.20503330866025166

In [482]:
global_churn - churn__with_partner 
# positive means less likely to churn, as that means it is less than global churn rate

0.06493474245795922

In [483]:
churn_no_partner = churn_male = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

0.3298090040927694

In [484]:
global_churn - churn_no_partner # Negative means higher likelyhood of churn, as that means it is higher than the global churn

-0.05984095297455855

Churn is lower if with partner and higher if no partner

**Risk Ratio**

In [485]:
churn_no_partner / global_churn # ~22% higher risk

1.2216593879412643

In [486]:
churn_with_partner / global_churn # ~26% lower risk

0.7594724924338315

SELECT<br>
&nbsp;&nbsp;&nbsp;gender,<br>
&nbsp;&nbsp;&nbsp;AVG(churn),<br>
&nbsp;&nbsp;&nbsp;AVG(churn),<br>
&nbsp;&nbsp;&nbsp;AVG(churn) - global_churn As diff,<br>
&nbsp;&nbsp;&nbsp;AVG(chrun) / global_churn As risk<br>
FROM<br>
&nbsp;&nbsp;&nbsp;data<br>
GROUP BY<br>
&nbsp;&nbsp;&nbsp;gender;<br>

In [487]:
from IPython.display import display

In [488]:
for c in categorical:
  df_group =df_full_train.groupby(c).churn.agg(['mean', 'count'])
  df_group['diff'] = df_group['mean'] - global_churn
  df_group['risk'] = df_group['mean'] / global_churn
  display(df_group)
  print()

,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208


,mean,count,diff,risk
partner,,,,
no,0.329809,2932,0.059841,1.221659
yes,0.205033,2702,-0.064935,0.759472


,mean,count,diff,risk
dependents,,,,
no,0.313760,3968,0.043792,1.162212
yes,0.165666,1666,-0.104302,0.613651


,mean,count,diff,risk
phoneservice,,,,
no,0.241316,547,-0.028652,0.893870
yes,0.273049,5087,0.003081,1.011412


...

,mean,count,diff,risk
streamingtv,,,,
no,0.342832,2246,0.072864,1.269897
no_internet_service,0.077805,1221,-0.192163,0.288201
yes,0.302723,2167,0.032755,1.121328


,mean,count,diff,risk
streamingmovies,,,,
no,0.338906,2213,0.068938,1.255358
no_internet_service,0.077805,1221,-0.192163,0.288201
yes,0.307273,2200,0.037305,1.138182


,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


# 3.6 Feature Importance: Mutual Importance

https://en.wikipedia.org/wiki/Mutual_information

In [489]:
from sklearn.metrics import mutual_info_score as mis

In [490]:
mis(df_full_train.churn, df_full_train.contract) # higher number, more important

0.0983203874041556

In [491]:
mis(df_full_train.gender, df_full_train.churn) # lower number less important

0.0001174846211139946

In [492]:
mis(df_full_train.churn, df_full_train.partner)

0.009967689095399745

In [493]:
def mutual_info_churn_score(series):
  return mis(series, df_full_train.churn)
# this function allows us to compare all the colums to the target variable. To determine the degree of affect, see above.

In [494]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False) # Sorted so highest first

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

# 3.7 Feature Importance: Correlation

https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

https://www.investopedia.com/terms/c/correlationcoefficient.asp

In [495]:
df_full_train.tenure.max()

72

In [496]:
df_full_train[numerical].corrwith(df_full_train.churn) # Add .abs() if we just wanted the number without direction

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

Negative correlation means that the higher the number the more it negatively affects the target value. 

In [499]:
df_full_train[df_full_train.tenure <=2].churn.mean()

0.5953420669577875

In [500]:
df_full_train[df_full_train.tenure > 2].churn.mean() # Notice the moderate churn despite the high negative correlation above, this indicates more investigation is needed.

0.22478269658378816

In [502]:
df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure <= 12)].churn.mean() # High turnover for 2-12 months

0.3994413407821229

In [503]:
df_full_train[df_full_train.tenure > 12].churn.mean() # Once over 12 monts, churn is low

0.17634908339788277

A target for tenure would be to get customers past the 12 month mark.

The same investigation is needed for the other variables. 

In [506]:
# Monthly Charges
monthly_under_20 = df_full_train[df_full_train.monthlycharges <=20].churn.mean()
monthly_between_20_50 = df_full_train[(df_full_train.monthlycharges >20) & (df_full_train.monthlycharges <=50) ].churn.mean()
monthly_over_50 = df_full_train[df_full_train.monthlycharges >50].churn.mean()
monthly_under_20, monthly_between_20_50, monthly_over_50

(0.08795411089866156, 0.18340943683409436, 0.32499341585462205)

In [507]:
# Total Charges
total_under_20 = df_full_train[df_full_train.totalcharges <=20].churn.mean()
total_between_20_50 = df_full_train[(df_full_train.totalcharges >20) & (df_full_train.totalcharges <=50) ].churn.mean()
total_over_50 = df_full_train[df_full_train.totalcharges>50].churn.mean()
total_under_20, total_between_20_50, total_over_50


(0.3108108108108108, 0.45089285714285715, 0.2618065967016492)

Notice Total Charges does not follow the same growth as Monthly Charges. This may be due to other factors such as partners being on the same plan which reduces churn while increasin total bill. 

**One-hot Encoding**

Use Scikit-Learn to encode categorical features

In [534]:
from sklearn.feature_extraction import DictVectorizer

In [535]:
df_train[['gender','contract']].loc[:10]

,gender,contract
0,female,two_year
1,male,month-to-month
2,female,month-to-month
3,female,month-to-month
4,female,two_year
5,male,month-to-month
6,male,month-to-month
7,female,month-to-month
8,female,two_year
9,female,month-to-month


In [536]:
dicts = df_train[['gender','contract']].loc[:10].to_dict(orient='records') 
dicts
#(orient='records') creates dictionay by row rather than the column

[{'gender': 'female', 'contract': 'two_year'},
 {'gender': 'male', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'two_year'},
 {'gender': 'male', 'contract': 'month-to-month'},
 {'gender': 'male', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'two_year'},
 {'gender': 'female', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'two_year'}]

In [537]:
dv = DictVectorizer(sparse=False) # Must set variable, aliasing does not provide the same functionality. 
# Sparse matrix is used when there are a high number of 0s, we have turned it off

In [538]:
dv.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


AttributeError: ignored

In [529]:
dv.fit(dicts)

DictVectorizer(sparse=False)

In [530]:
dv.transform(dicts)

array([[0., 1., 1., 0.],
       [1., 0., 0., 1.],
       [1., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 1., 1., 0.],
       [1., 0., 0., 1.],
       [1., 0., 0., 1.],
       [1., 0., 1., 0.],
       [0., 1., 1., 0.],
       [1., 0., 1., 0.],
       [0., 1., 1., 0.]])